### 重置摄像头

In [1]:
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'

[sudo] password for jetson: 


### 初始化

In [5]:
import torch
import torchvision
from torch2trt import torch2trt
from torch2trt import TRTModule
TASK="test"
CATEGORIES = ['apex']
device = torch.device('cuda')

# 加载路径跟随模型
modelroad = torchvision.models.resnet18(pretrained=False)
modelroad.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
modelroad = modelroad.cuda().eval().half()
modelroad.load_state_dict(torch.load(TASK+'/path/model.pth'))

data = torch.zeros((1, 3, 224, 224)).cuda().half()

modelroad_trt = torch2trt(modelroad, [data], fp16_mode=True)

torch.save(modelroad_trt.state_dict(), TASK+'/road_model_trt.pth')

modelroad_trt = TRTModule()
modelroad_trt.load_state_dict(torch.load(TASK+'/road_model_trt.pth'))

# 加载交通信号分类模型
# from jetbot import ObjectDetector

# model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

from PIL import Image
import sys
import os
import urllib
import tensorflow.contrib.tensorrt as trt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import numpy as np
import time

outputNames=['detection_boxes', 'detection_classes', 'detection_scores', 'num_detections']
inputNames=['image_tensor']

from tensorflow.python.platform import gfile

tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True

tf_sess = tf.Session(config=tf_config)
with gfile.FastGFile('./model_tf/v2/trt_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf_sess.graph.as_default()
    tf.import_graph_def(graph_def, name='') # 导入计算图
tf_input = tf_sess.graph.get_tensor_by_name(input_names[0] + ':0')
tf_scores = tf_sess.graph.get_tensor_by_name('detection_scores:0')
tf_boxes = tf_sess.graph.get_tensor_by_name('detection_boxes:0')
tf_classes = tf_sess.graph.get_tensor_by_name('detection_classes:0')
tf_num_detections = tf_sess.graph.get_tensor_by_name('num_detections:0')


from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

初始化摄像头

In [6]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=300, height=300, capture_fps=50)

开始跑

In [7]:
from utils import preprocess
import numpy as np
import threading
import ipywidgets
import time

car.steering_gain = -0.55
car.steering_offset = -0.13
car.throttle_gain = 0.5
# car.throttle_gain = 0.5
# car.throttle = 0.25
speed_widget = ipywidgets.IntText(description='速度', value=20)
run_button = ipywidgets.Button(description='启动')
state = 1


        
    

# 运行的主循环
def run(car,camera,speed_widget,run_button):
    global state
    loop = 0
    # 红绿灯信号 1表示要停5秒
    signalRGB = 0
    #路口方向  0：直行，-1左转  1右转
    chooseDirect = 2 
    forbiddenDirect ={}
    enableDirect ={}
    #是否不再路口 0表示进入路口
    inRoad = 0
    # 确认交通信号的阈值
    condition = 0.9  
# string_val: "corner"
# string_val: "no left turn"
# string_val: "no right turn"
# string_val: "no direct line"
# string_val: "no entry"
# string_val: "left turn"
# string_val: "right turn"
# string_val: "direct line"
# string_val: "signal"
# string_val: "footway"
    # 路口方向未定，判断交通信号
    def noLeftTurn(){
        forbiddenDirect.append(-1)
    }
    def noRightTurn(){
        forbiddenDirect.append(1)
    }
    def noDirectTurn(){#和no entry相同
        forbiddenDirect.append(0)
    }
    def leftTurn(){
        chooseDirect = -1
    }
    def rightTurn(){
        chooseDirect = 1
    }
    def directTurn(){
        chooseDirect = 0
    }
    def footway(){
        #暂不处理
    }
    def signal(){
        if (signalRGB==0):
            signalRGB==1
    }
    def corner(){
        #暂不处理
    }            
    # 判断路口并处理
    def checkCrossing(output): 
        //todo 根据corner的位置确认路口
        if (output[CROSS_CATEGORIES.index('十字路口')]>condition):
            enableDirect = {-1,1,0}
        elif (output[CROSS_CATEGORIES.index('左三叉路口')]>condition):
            enableDirect = {-1,0}
        elif (output[CROSS_CATEGORIES.index('右三叉路口')]>condition):
            enableDirect = {1,0}
        elif (output[CROSS_CATEGORIES.index('左右三叉路口')]>condition):
            enableDirect = {-1,1} 
        elif
            inRoad = 1
            
           
    # 根据可选方向，禁行方向和路口方向确定方向。如果SRGB=1，停车，sleep 5秒，设置SRGB=2。 直接设置转向。
    def inCrossing():
        if signalRGB == 1:
            # 红绿灯 停5秒
            signalRGB = 2
            car.throttle = -0.1
            time.sleep(0.5)
            car.throttle = 0
            time.sleep(4.5)
            car.throttle = speed_widget.value*0.01
        if (chooseDirect==2):
            #确定可以行走的方向
            enableDirect=enableDirect-forbiddenDirect
            if (len(enableDirect)!=0):
                #确定方向
                chooseDirect=enableDirect[0]
        if (chooseDirect!=2):
            car.steering=0.45*chooseDirect
            
    signalsFunc=[corner(),noLeftTurn(),noRightTurn(),noDirectTurn(),noDirectTurn(),leftTurn(),rightTurn(),directTurn(),signal(),footway()] 
    while True:
        if (state == 1):
            car.throttle = 0
            car.steering = 0
            time.sleep(0.5)
        else:
            throttle = 0
            # 准备摄像头数据
            image = camera.read()
            image = preprocess(image).half()
            detections = model(image)
            if (loop%5==0) and (inRoad == 1):
                # 识别交通信号 每5帧处理一次(进入路口后就不处理)
                scores, boxes, classes, num_detections = tf_sess.run([tf_scores, tf_boxes, tf_classes, tf_num_detections], feed_dict={
                    tf_input: image[None, ...]
                })

                boxes = boxes[0] # index by 0 to remove batch dimension
                scores = scores[0]
                classes = classes[0]
                num_detections = num_detections[0]
                for index,scores in enumerate(scores):
                    if scores>0.6:
                        box = boxes[index]
                        if (classes[index]==1):
                            signalsFunc[classes[index]]()
                        elif (classes[index]==9 ) and (box[2]-box[1]>0.15):
                            signalsFunc[classes[index]]()
                        elif (classes[index]>=2) and (classes[index]<=8) and (box[2]-box[1]>0.2) and (chooseDirect==2):  
                            signalsFunc[classes[index]]();
            
                
                if (len(enableDirect)==0):
                    checkCrossing(output)
                    
                # 进入路口
                if (len(enableDirect)!=0):
                    inRoad = 0
                    inCrossing()
                    
                # 没有进入路口      
                if (len(enableDirect)==0) or (inRoad == 1):
                    inRoad = 1
                    chooseDirect = 2 
                    forbiddenDirect ={}
                    enableDirect ={}                    
                    
            loop += 1
            #在道路上才由road模型决定方向
            if inRoad == 1:
                output = modelroad_trt(image).detach().cpu().numpy().flatten()
                x = float(output[0])
                car.steering = x 
                car.throttle = speed_widget.value*0.01
        
        
execute_thread = threading.Thread(target=run, args=(car,camera,speed_widget,run_button))

def runclick(c):
    global state
    print(run_button.description)
    if run_button.description == '启动':
        run_button.description = '停止'
        state = 0
    else:
        run_button.description = '启动'
        state = 1

run_button.on_click(runclick)

data_collection_widget = ipywidgets.HBox([
    run_button,
    speed_widget
])

display(data_collection_widget)
execute_thread.start()
